In [1]:
from pypdf import PdfReader

c:\Users\swamp\OneDrive\Escritorio\CatchAI\catchAI\lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [ ]:
def upload_pdf(pdf):
    reader = PdfReader(pdf)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

def pdf_to_txt(text, pdf_name):
    with open(f"pdf/{pdf_name}.txt", "w", encoding="utf-8") as file:
        file.write(text)
    print(f"Texto guardado en pdf/{pdf_name}.txt")
    return text

def read_text(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
    return text

    

In [3]:
import os
from pypdf import PdfReader

pdf_dir = "pdf"
pdf_files = [f for f in os.listdir(pdf_dir) if f.lower().endswith(".pdf")]

for file_name in pdf_files:
    file_path = os.path.join(pdf_dir, file_name)
    print(f"Procesando: {file_path}")
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    pdf_name = os.path.splitext(file_name)[0]
    pdf_to_txt(text, pdf_name)

Procesando: pdf\Conceptos y temas claves.pdf
Texto guardado en pdf/Conceptos y temas claves.txt


In [4]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter

txt_dir = "pdf"
txt_files = [f for f in os.listdir(txt_dir) if f.lower().endswith(".txt")]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=750,
    chunk_overlap=int(750*0.15),
    length_function=len,
)

all_chunks = []
all_chunk_ids = []

for file_name in txt_files:
    file_path = os.path.join(txt_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
    chunks = text_splitter.create_documents([text])
    for idx, doc in enumerate(chunks):
        chunk_id = f"{file_name}_chunk{idx+1}"
        all_chunks.append(doc.page_content)
        all_chunk_ids.append(chunk_id)

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
all_vectors = embeddings.embed_documents(all_chunks)

C:\Users\swamp\AppData\Local\Temp\ipykernel_24644\428911517.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
d:\CatchAI\catchAI\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Guardar en base vectorial Chroma
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_texts(
    all_chunks,
    embeddings,
    ids=all_chunk_ids,
    persist_directory="./chroma_db"
)
vectorstore.persist()

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
C:\Users\swamp\AppData\Local\Temp\ipykernel_24644\2151559639.py:10: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [22]:
# Convierte la consulta en embedding
user_query = "¿Qué experiencia tiene Tomas en Python?"
query_vector = embeddings.embed_query(user_query)

# Busca los chunks más similares en Chroma
results = vectorstore.similarity_search_by_vector(query_vector, k=5)

relevant_chunks = [
    result.page_content for result in results
    if "Python" in result.page_content or "programación" in result.page_content
]

In [23]:
context = "\n".join(relevant_chunks)
prompt = (
    f"Contexto:\n{context}\n\n"
    f"Pregunta:\n{user_query}\n\n"
    "Responde de forma clara y específica sobre la experiencia en Python."
)

In [24]:
print(prompt)

Contexto:
•Seidentificanáreasdemejora,comolanecesidaddeunentrenamientomásminuciosoparaelmodeloyconsiderar losvaloresentregadospor”Botometer”paraevitarfalsospositivosynegativosenlaclasificación Educación UniversidadDiegoPortales Santiago,Chile TITULADO DE INGENIERÍA CIVIL EN INFORMÁTICA Y TELECOMUNICACIONES Mar.2019-Ene.2024 •Cursosrelevantes:IA,DataScience,SistemasDistribuidos,SistemasdeProcesamientoparaBigData Habilidades Lenguajes Python,SQL,C/C++,Ruby MachineLearning/CienciadeDatos Pandas,Numpy,Scikit-learn,Tensorflow,Matplotlib BasesdeDatos PostgreSQL,SQLServer Cloud AWSS3,Docker,Git,GCP Idiomas EspañolNativo,Inglés Otros PowerBI,Unix 29 DE JULIO DE 2025 TOMÁS FUENTES ARAYA · CURRICULUM VITAE
árbolesdedecisión.
•Selograunaprecisióndel88%,evaluandoelimpactodecaracterísticastextualesydecomportamiento.
•Seidentificanáreasdemejora,comolanecesidaddeunentrenamientomásminuciosoparaelmodeloyconsiderar
losvaloresentregadospor”Botometer”paraevitarfalsospositivosynegativosenlaclasificación
Ed